# Lib

In [74]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import sys
import os
import glob

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters

from ipywidgets import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

18/11/2022  19:16    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
21/11/2022  23:51         1.007.073 1.EA.ipynb
22/11/2022  22:16            77.139 2.Feature_engineering.ipynb
22/11/2022  22:31            28.749 3.Stability_var.ipynb
22/11/2022  23:54           128.740 4.Modeling.ipynb
21/11/2022  19:26    <DIR>          catboost_info
               4 arquivo(s)      1.241.701 bytes
               4 pasta(s)   358.015.369.216 bytes dispon�veis


# Reading

In [3]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [4]:
!dir .\\..\data

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\data

22/11/2022  22:27    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/08/2021  12:08        26.677.890 dados.csv
22/11/2022  23:52        13.834.060 EA.pbix
22/11/2022  22:34            39.018 iv_woe_by_week.parquet
09/11/2022  10:35            54.091 MySQL_localhost_connection.pbix
24/11/2022  23:31         3.891.056 test_dataset.parquet
24/11/2022  23:31         8.711.592 train_dataset.parquet
24/11/2022  23:29        14.580.572 train_test_dataset_tranformed.parquet
24/11/2022  23:31         2.094.822 validation_dataset.parquet
               8 arquivo(s)     69.883.101 bytes
               2 pasta(s)   358.015.352.832 bytes dispon�veis


In [125]:
def create_woe_columns(df):

    ## criando uma copia
    dff = df.copy()
    ## Selecionando as colunas para serem usadas
    colunas_orig_and_week = utils_ml.exclude_words( dff.columns.tolist(), ["cat","bin","WoE","produto","fecha","data","hora","week","fraude"] ) + ['categoria_produto']
    ## Iterando para adicionar as colunas WoE
    for n,variable in enumerate(colunas_orig_and_week, start=1):

        optb = joblib.load(f".\..\src\\features\\pipeline_binning__{variable}__.pkl")

        dff = utils_ml.encode_woe_var_transform(dff, variable, optb )


    return dff


In [193]:
## test-train dataset
df = pd.read_parquet(f"{parameters.data_path}\\train_test_dataset_tranformed.parquet")
df['data'] = pd.to_datetime(df['data'])

## validation Dataset
df_validation = pd.read_parquet(f"{parameters.data_path}\\validation_dataset.parquet")
df_validation['data'] = pd.to_datetime(df_validation['data'])
## Creating WoE columns
df_validation = create_woe_columns(df_validation)

In [130]:
features_bins_woe = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
## Excluding variable 'score' (score of Meli model) and 'o_obj' (with 72.5% of missing values))
features_bins_woe = utils_ml.exclude_words(features_bins_woe, ["score","o_obj"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins',
 'data_bins']

# Model

## Logistic regression

In [15]:
df_2_model = df[features_bins_woe + ["fraude"]]

### Dumming features

In [19]:
# from sklearn import preprocessing

# le = preprocessing.LabelEncoder()

# cat_vars = features_bins_woe

# df_2_model["pais_hot_encode"] = le.fit_transform(df_2_model['pais_bins'])


# X = data.iloc[:, 0:4]
# y = data.iloc[:, 5]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

# df_2_model[["pais_hot_encode",'pais_bins']].value_counts()

In [88]:
def transform( df , label = "fraude", drop_first = True):

    features = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
    features = utils_ml.exclude_words(features, ["score","o_obj"])

    ls_df_dummies = []

    for var in features:

        ## creating dummy to variable
        df_dummy = pd.get_dummies( df[var] , prefix=var, drop_first = drop_first)
        ls_df_dummies.append(df_dummy)


    df_2_model_dum = pd.concat(ls_df_dummies + [ df[label] ], axis=1)

    return df_2_model_dum

In [89]:
df_2_model_dum = transform( df_2_model ,  label = "fraude")

In [90]:
def transform_2_model(df ,  label ="fraude", cols_model = df_2_model_dum.columns.tolist() ):

    ## tirando a coluna label caso esteja na lista
    cols_model = utils_ml.exclude_words(cols_model, [label] )
    ## Transformando para dummies
    df_marco_model_dum = transform(df = df, label ="fraude")

    ## Preenchendo colunas faltantes
    dummies_faltantes = list(   set(cols_model) - set(df_marco_model_dum.columns.to_list()) )

    if len(dummies_faltantes)>0:

        for dummi_falt in dummies_faltantes:

            df_marco_model_dum[dummi_falt] = 0

    return df_marco_model_dum[cols_model+[label]]

In [91]:
df_2_model_dum.columns

Index(['n_boolean_bins_1', 'p_boolean_bins_1', 'categoria_produto_bins_1', 'categoria_produto_bins_2', 'categoria_produto_bins_3', 'categoria_produto_bins_4', 'categoria_produto_bins_5', 'categoria_produto_bins_6', 'categoria_produto_bins_7', 'categoria_produto_bins_8', 'categoria_produto_bins_9', 'categoria_produto_bins_10', 'pais_bins_0', 'pais_bins_1', 'a_int_bins_1', 'b_float_bins_0', 'b_float_bins_1', 'b_float_bins_2', 'b_float_bins_3', 'b_float_bins_4', 'b_float_bins_5', 'c_float_bins_0', 'c_float_bins_1', 'd_float_bins_0', 'd_float_bins_1', 'd_float_bins_2', 'd_float_bins_3', 'd_float_bins_4', 'd_float_bins_5', 'd_float_bins_6', 'e_float_bins_1', 'e_float_bins_2', 'f_float_bins_0', 'f_float_bins_1', 'f_float_bins_2', 'f_float_bins_3', 'f_float_bins_4', 'f_float_bins_5', 'f_float_bins_6', 'f_float_bins_7', 'f_float_bins_8', 'f_float_bins_9', 'h_float_bins_1', 'h_float_bins_2', 'h_float_bins_3', 'h_float_bins_4', 'h_float_bins_5', 'k_float_bins_1', 'k_float_bins_2', 'k_float_bins_

### Train and test samples

In [23]:
from sklearn.model_selection import train_test_split

X = df_2_model_dum.drop(['fraude'], axis=1)
y = df_2_model_dum['fraude']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

In [24]:
y.value_counts(normalize=True)[0]/y.value_counts(normalize=True)[1]

18.727591463414633

In [25]:
# X_train.columns.tolist()

### Trainning

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# lr = LogisticRegression(random_state=0,solver='newton-cg', fit_intercept=True, penalty='l2' )

# #Setting the range for class weights
# weights = np.linspace(0.0,0.99,200)

# #Creating a dictionary grid for grid search
# param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}

# #Fitting grid search to the train data with 5 folds
# gridsearch = GridSearchCV(estimator= lr, 
#                           param_grid= param_grid,
#                           cv=StratifiedKFold(), 
#                           n_jobs=-1, 
#                           scoring='recall', 
#                           verbose=2).fit(X_train, y_train)

## Ploting the score for different values of weight
# sns.set_style('whitegrid')
# plt.figure(figsize=(12,8))
# weigh_data = pd.DataFrame({ 'score': gridsearch.cv_results_['mean_test_score'], 'weight': (1-weights)})
# sns.lineplot(weigh_data,  x='weight', y='score')
# plt.xlabel('Weight for class 1')
# plt.ylabel('F1 score')
# plt.xticks([round(i/10,1) for i in range(0,11,1)])
# plt.title('Scoring for different class weights', fontsize=24)

In [27]:
# LogisticRegression??

In [28]:
clf = LogisticRegression( random_state=0, solver='saga',l1_ratio = 0.1, fit_intercept=True, penalty='elasticnet', class_weight = "balanced" )

model = clf.fit(X_train, y_train )

c:\Users\caanp\miniconda3\envs\venv_ml\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [29]:
df_2_model['probabilty'] = model.predict_proba(X)[:,1]

C:\Users\caanp\AppData\Local\Temp\ipykernel_35136\2283094811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_model['probabilty'] = model.predict_proba(X)[:,1]


In [30]:
colunas_orig = utils_ml.exclude_words( df.columns.tolist(), ["bin","cat","WoE","fecha",'data','hora'] ) + ['categoria_produto']
colunas_orig

['a_int',
 'b_float',
 'c_float',
 'd_float',
 'e_float',
 'f_float',
 'pais',
 'h_float',
 'produto',
 'k_float',
 'l_float',
 'm_float',
 'n_boolean',
 'o_obj',
 'p_boolean',
 'monto',
 'score',
 'fraude',
 'categoria_produto']

In [31]:
df_ivs_ls = []

for variable in colunas_orig:

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype=='O' or df[variable].dtype=='category' else 'numerical'

    df_iv = utils_ml.WOE_IV(df = df , variable = variable, dtype_ = dtype_ , label = label, monotonic= True, optimal_binning=True)[[ 'variable' , 'IV_var', 'KS']].drop_duplicates()

    df_ivs_ls.append(df_iv)

pd.concat(df_ivs_ls).sort_values("IV_var", ascending=False)

,variable,IV_var,KS
0,produto,6.575706,89.262752
0,categoria_produto,1.541493,40.920152
0,o_obj,1.111551,45.338385
0,score,0.989984,42.673514
0,f_float,0.443467,27.653106
0,l_float,0.428472,25.054252
0,n_boolean,0.363338,22.847129
0,m_float,0.341464,23.071558
0,p_boolean,0.246970,24.429978
0,d_float,0.170680,16.498234


In [32]:
coef_df = pd.DataFrame( {"Cols": model.feature_names_in_,  "coef": model.coef_[0] })
coef_df.sort_values("coef", ascending=False)

,Cols,coef
11,categoria_produto_bins_10,7.104002
10,categoria_produto_bins_9,6.695268
9,categoria_produto_bins_8,5.704992
8,categoria_produto_bins_7,5.088206
7,categoria_produto_bins_6,4.883429
5,categoria_produto_bins_4,4.431704
6,categoria_produto_bins_5,4.266193
2,categoria_produto_bins_1,3.767609
4,categoria_produto_bins_3,3.725608
3,categoria_produto_bins_2,3.045451


### comparing with Meli model

> **modelo MELI**

In [33]:
probability = "score"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,12942,"[0, 8]",368,12574,6560,122853,0,0,5.609756,10.234996,5.609756,10.234996,42.087631,1.000000,0.050690,0.000000,0.096490,0.210725
1,12941,"[8, 18]",612,12329,6192,110279,12574,368,9.329268,10.035571,14.939024,20.270567,42.087631,0.943902,0.053163,0.102350,0.100658,0.216942
2,12941,"[18, 28]",404,12537,5580,97950,24903,980,6.158537,10.204879,21.097561,30.475446,42.087631,0.850610,0.053897,0.202706,0.101372,0.214996
3,12941,"[28, 39]",43,12898,5176,85413,37440,1384,0.655488,10.498726,21.753049,40.974172,42.087631,0.789024,0.057137,0.304754,0.106558,0.221520
4,12942,"[39, 49]",89,12853,5133,72515,50338,1427,1.356707,10.462097,23.109756,51.436269,42.087631,0.782470,0.066106,0.409742,0.121912,0.247045
5,12941,"[49, 59]",166,12775,5044,59662,63191,1516,2.530488,10.398606,25.640244,61.834876,42.087631,0.768902,0.077953,0.514363,0.141554,0.277307
6,12941,"[59, 69]",289,12652,4878,46887,75966,1682,4.405488,10.298487,30.045732,72.133363,42.087631,0.743598,0.094234,0.618349,0.167270,0.312672
7,12941,"[69, 79]",710,12231,4589,34235,88618,1971,10.823171,9.955801,40.868902,82.089163,42.087631,0.699543,0.118200,0.721334,0.202230,0.352653
8,12941,"[79, 89]",1157,11784,3879,22004,100849,2681,17.637195,9.591951,58.506098,91.681115,42.087631,0.591311,0.149867,0.820892,0.239127,0.372101


> **Logistic Regression**

In [34]:
probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_2_model, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,12942,"[2.5941471488753014e-07, 8.981970577030873e-05]",17,12925,6560,122853,0,0,0.259146,10.520704,0.259146,10.520704,41.027811,1.000000,0.050690,0.000000,0.096490,0.210725
1,12941,"[8.983394814726988e-05, 0.00039888125921671376]",46,12895,6543,109928,12925,17,0.701220,10.496284,0.960366,21.016988,41.027811,0.997409,0.056177,0.105207,0.106363,0.229240
2,12941,"[0.00039888483899318803, 0.0015044816252378094]",125,12816,6497,97033,25820,63,1.905488,10.431980,2.865854,31.448967,41.027811,0.990396,0.062755,0.210170,0.118031,0.250328
3,12941,"[0.001505180338333732, 0.004708228187324111]",268,12673,6372,84217,38636,188,4.085366,10.315580,6.951220,41.764548,41.027811,0.971341,0.070340,0.314490,0.131180,0.272706
4,12942,"[0.004708228187324111, 0.012574367011547544]",406,12536,6104,71544,51309,456,6.189024,10.204065,13.140244,51.968613,41.027811,0.930488,0.078611,0.417645,0.144974,0.293778
5,12941,"[0.012574598320490751, 0.029807626396258717]",524,12417,5698,59008,63845,862,7.987805,10.107201,21.128049,62.075814,41.027811,0.868598,0.088060,0.519686,0.159908,0.313263
6,12941,"[0.029810171113678733, 0.06973395306574999]",651,12290,5174,46591,76262,1386,9.923780,10.003826,31.051829,72.079640,41.027811,0.788720,0.099952,0.620758,0.177420,0.331645
7,12941,"[0.06973569023421473, 0.16766613555104856]",839,12102,4523,34301,88552,2037,12.789634,9.850797,43.841463,81.930437,41.027811,0.689482,0.116500,0.720796,0.199321,0.347581
8,12941,"[0.16773484232523278, 0.43057555300815936]",1179,11762,3684,22199,100654,2876,17.972561,9.574044,61.814024,91.504481,41.027811,0.561585,0.142333,0.819304,0.227106,0.353395


>> **Logistic Regression in test sample**

In [36]:
df_test = pd.DataFrame()
df_test['fraude'] = y_test
df_test['probabilty'] = model.predict_proba(X_test)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_test, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,3883,"[2.5941471488753014e-07, 9.176144194691357e-05]",4,3879,1968,36856,0,0,0.203252,10.524745,0.203252,10.524745,41.631648,1.000000,0.050690,0.000000,0.096490,0.210725
1,3882,"[9.176206856358303e-05, 0.0004107213096158194]",22,3860,1964,32977,3879,4,1.117886,10.473193,1.321138,20.997938,41.631648,0.997967,0.056209,0.105247,0.106424,0.229370
2,3882,"[0.00041081624564576766, 0.0015460343954167663]",30,3852,1942,29117,7739,26,1.524390,10.451487,2.845528,31.449425,41.631648,0.986789,0.062526,0.209979,0.117601,0.249416
3,3883,"[0.0015462735356701672, 0.004848566276561534]",76,3807,1912,25265,11591,56,3.861789,10.329390,6.707317,41.778815,41.631648,0.971545,0.070354,0.314494,0.131206,0.272761
4,3882,"[0.004848646845599397, 0.01261410719750558]",114,3768,1836,21458,15398,132,5.792683,10.223573,12.500000,52.002388,41.631648,0.932927,0.078819,0.417788,0.145357,0.294552
5,3882,"[0.012621175176295334, 0.029909564509840753]",157,3725,1722,17690,19166,246,7.977642,10.106903,20.477642,62.109290,41.631648,0.875000,0.088708,0.520024,0.161085,0.315570
6,3883,"[0.029912538282562205, 0.0694696103137759]",213,3670,1565,13965,22891,403,10.823171,9.957673,31.300813,72.066963,41.631648,0.795224,0.100773,0.621093,0.178878,0.334373
7,3882,"[0.06947550024386918, 0.16579268760269825]",272,3610,1352,10295,26561,616,13.821138,9.794877,45.121951,81.861841,41.631648,0.686992,0.116081,0.720670,0.198604,0.346329
8,3882,"[0.16580108842599098, 0.42559220096359923]",339,3543,1080,6685,30171,888,17.225610,9.613089,62.347561,91.474929,41.631648,0.548780,0.139086,0.818618,0.221925,0.345335


>> **Logistic Regression in validation sample**

In [131]:
df_validation_2_model = transform_2_model(df_validation)
df_validation_2_model['probabilty'] = model.predict_proba( df_validation_2_model.drop("fraude", axis=1) )[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation( df_validation_2_model, probability, label, quantil )

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,2059,"[3.169700519414198e-07, 7.190569576131649e-05]",40,2019,940,19647,0,0,4.255319,10.276378,4.255319,10.276378,24.413055,1.000000,0.045660,0.000000,0.087332,0.193042
1,2059,"[7.190626006968409e-05, 0.0003227126720702342]",44,2015,900,17628,2019,40,4.680851,10.256019,8.936170,20.532397,24.413055,0.957447,0.048575,0.102764,0.092459,0.201902
2,2058,"[0.00032307086095565893, 0.001204543430557559]",62,1996,856,15613,4034,84,6.595745,10.159312,15.531915,30.691709,24.413055,0.910638,0.051976,0.205324,0.098340,0.211577
3,2059,"[0.0012056146815159258, 0.003804756291980722]",52,2007,794,13617,6030,146,5.531915,10.215300,21.063830,40.907009,24.413055,0.844681,0.055097,0.306917,0.103446,0.218480
4,2059,"[0.0038090382854005264, 0.010199539522690326]",81,1978,742,11610,8037,198,8.617021,10.067695,29.680851,50.974704,24.413055,0.789362,0.060071,0.409070,0.111646,0.230263
5,2058,"[0.010204878846394127, 0.02392857866620654]",71,1987,661,9632,10015,279,7.553191,10.113503,37.234043,61.088207,24.413055,0.703191,0.064218,0.509747,0.117689,0.235181
6,2059,"[0.023933739362755183, 0.05613953090427404]",89,1970,590,7645,12002,350,9.468085,10.026976,46.702128,71.115183,24.413055,0.627660,0.071645,0.610882,0.128610,0.245936
7,2058,"[0.05617469728131022, 0.13715917001770603]",107,1951,501,5675,13972,439,11.382979,9.930269,58.085106,81.045452,24.413055,0.532979,0.081120,0.711152,0.140809,0.252114
8,2059,"[0.13723207060102036, 0.3682184970915063]",116,1943,394,3724,15923,546,12.340426,9.889551,70.425532,90.935003,24.413055,0.419149,0.095678,0.810455,0.155793,0.250063


## Catboost

In [20]:
# !pip install catboost

In [37]:
import catboost as cb

### Train and test samples

In [176]:
def transform_2_cat_model(df):

    cat_features = [x for x in utils_ml.list_subset_words( df.columns.tolist(), ['cat']) if "WoE" not in x and "bin" not in x and x!='categoria_produto']
    cat_features = utils_ml.exclude_words(cat_features, ["score","o_obj","data"])

    try:
        df_2_model_cat = df[ cat_features + ["fraude"] ]
    except:
        df_2_model_cat = df[ cat_features ]


    return df_2_model_cat

In [169]:
cat_features = [x for x in utils_ml.list_subset_words( df.columns.tolist(), ['cat']) if "WoE" not in x and "bin" not in x and x!='categoria_produto']
cat_features = utils_ml.exclude_words(cat_features, ["score","o_obj","data"])


df_2_model_cat = df[cat_features + ["fraude"]]


X = df_2_model_cat.drop(['fraude'], axis=1)
y = df_2_model_cat['fraude']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

In [150]:
cat_features

['n_boolean_cat',
 'p_boolean_cat',
 'categoria_produto_cat',
 'pais_cat',
 'a_int_cat',
 'b_float_cat',
 'c_float_cat',
 'd_float_cat',
 'e_float_cat',
 'f_float_cat',
 'h_float_cat',
 'k_float_cat',
 'l_float_cat',
 'm_float_cat',
 'monto_cat']

### Trainning

In [151]:
model_cat_def = cb.CatBoostClassifier(
                           l2_leaf_reg = 3.0,
                           scale_pos_weight = 19,
                           verbose=False,
                           thread_count = 8,
                           random_seed = 0
                          )


modelo_cat = model_cat_def.fit(X_test , y_test, cat_features =  cat_features  )

In [152]:
coef = pd.DataFrame( {"features":modelo_cat.feature_names_,"coef":modelo_cat.feature_importances_})
coef = coef.sort_values("coef", ascending=False).reset_index(drop=True).query("coef>0")
coef

,features,coef
0,categoria_produto_cat,27.676825
1,f_float_cat,11.825328
2,l_float_cat,9.388799
3,m_float_cat,7.850514
4,monto_cat,6.397801
5,k_float_cat,6.257384
6,e_float_cat,5.864204
7,h_float_cat,5.307048
8,b_float_cat,4.989970
9,d_float_cat,4.665863


### comparing with Meli model

> **modelo MELI**

In [154]:
probability = "score"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,12942,"[0, 8]",368,12574,6560,122853,0,0,5.609756,10.234996,5.609756,10.234996,42.087631,1.000000,0.050690,0.000000,0.096490,0.210725
1,12941,"[8, 18]",612,12329,6192,110279,12574,368,9.329268,10.035571,14.939024,20.270567,42.087631,0.943902,0.053163,0.102350,0.100658,0.216942
2,12941,"[18, 28]",404,12537,5580,97950,24903,980,6.158537,10.204879,21.097561,30.475446,42.087631,0.850610,0.053897,0.202706,0.101372,0.214996
3,12941,"[28, 39]",43,12898,5176,85413,37440,1384,0.655488,10.498726,21.753049,40.974172,42.087631,0.789024,0.057137,0.304754,0.106558,0.221520
4,12942,"[39, 49]",89,12853,5133,72515,50338,1427,1.356707,10.462097,23.109756,51.436269,42.087631,0.782470,0.066106,0.409742,0.121912,0.247045
5,12941,"[49, 59]",166,12775,5044,59662,63191,1516,2.530488,10.398606,25.640244,61.834876,42.087631,0.768902,0.077953,0.514363,0.141554,0.277307
6,12941,"[59, 69]",289,12652,4878,46887,75966,1682,4.405488,10.298487,30.045732,72.133363,42.087631,0.743598,0.094234,0.618349,0.167270,0.312672
7,12941,"[69, 79]",710,12231,4589,34235,88618,1971,10.823171,9.955801,40.868902,82.089163,42.087631,0.699543,0.118200,0.721334,0.202230,0.352653
8,12941,"[79, 89]",1157,11784,3879,22004,100849,2681,17.637195,9.591951,58.506098,91.681115,42.087631,0.591311,0.149867,0.820892,0.239127,0.372101


> **Catboost model**

In [155]:
df_2_model_cat['probabilty'] = modelo_cat.predict_proba(X)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_2_model_cat, probability, label, quantil)

C:\Users\caanp\AppData\Local\Temp\ipykernel_35136\240659597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_model_cat['probabilty'] = modelo_cat.predict_proba(X)[:,1]


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,12942,"[0.0006697300079497619, 0.016347987147811244]",10,12932,6560,122853,0,0,0.152439,10.526401,0.152439,10.526401,53.601133,1.000000,0.050690,0.000000,0.096490,0.210725
1,12941,"[0.01634802666749804, 0.026851872665988253]",16,12925,6550,109921,12932,10,0.243902,10.520704,0.396341,21.047105,53.601133,0.998476,0.056237,0.105264,0.106477,0.229485
2,12941,"[0.026852742171091955, 0.07689947083947994]",40,12901,6534,96996,25857,26,0.609756,10.501168,1.006098,31.548273,53.601133,0.996037,0.063112,0.210471,0.118703,0.251753
3,12941,"[0.07693159072265941, 0.17321333356522392]",166,12775,6494,84095,38758,66,2.530488,10.398606,3.536585,41.946880,53.601133,0.989939,0.071686,0.315483,0.133692,0.277928
4,12942,"[0.17321658873669915, 0.24325087748890342]",251,12691,6328,71320,51533,232,3.826220,10.330232,7.362805,52.277112,53.601133,0.964634,0.081496,0.419469,0.150295,0.304559
5,12941,"[0.2432545312537595, 0.3139790210527601]",321,12620,6077,58629,64224,483,4.893293,10.272439,12.256098,62.549551,53.601133,0.926372,0.093917,0.522771,0.170544,0.334099
6,12941,"[0.3139918960536157, 0.39571555401393543]",450,12491,5756,46009,76844,804,6.859756,10.167436,19.115854,72.716987,53.601133,0.877439,0.111195,0.625496,0.197377,0.368951
7,12941,"[0.39571592519184656, 0.509644986789141]",739,12202,5306,33518,89335,1254,11.265244,9.932195,30.381098,82.649182,53.601133,0.808841,0.136668,0.727170,0.233827,0.407752
8,12941,"[0.5096913767405151, 0.6825449608780024]",1249,11692,4567,21316,101537,1993,19.039634,9.517065,49.420732,92.166247,53.601133,0.696189,0.176448,0.826492,0.281540,0.438098


>> **Catboost in test sample**

In [180]:
df_test = pd.DataFrame()
df_test['fraude'] = y_test
df_test['probabilty'] = modelo_cat.predict_proba(X_test)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_test, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,3883,"[0.0006697300079497619, 0.016550776453516127]",0,3883,1968,36856,0,0,0.000000,10.535598,0.000000,10.535598,66.084076,1.000000,0.050690,0.000000,0.096490,0.210725
1,3882,"[0.01655564282137681, 0.026778551027002016]",0,3882,1968,32973,3883,0,0.000000,10.532885,0.000000,21.068483,66.084076,1.000000,0.056324,0.105356,0.106641,0.229837
2,3882,"[0.02678079770819631, 0.08015261140276171]",5,3877,1968,29091,7765,0,0.254065,10.519318,0.254065,31.587801,66.084076,1.000000,0.063363,0.210685,0.119175,0.252755
3,3883,"[0.08015807553200334, 0.17381496591690104]",5,3878,1963,25214,11642,5,0.254065,10.522032,0.508130,42.109833,66.084076,0.997459,0.072230,0.315878,0.134706,0.280037
4,3882,"[0.1738395458712417, 0.24320164611252032]",11,3871,1958,21336,15520,10,0.558943,10.503039,1.067073,52.612872,66.084076,0.994919,0.084056,0.421098,0.155015,0.314124
5,3882,"[0.2432370633360381, 0.3140274913733357]",33,3849,1947,17465,19391,21,1.676829,10.443347,2.743902,63.056219,66.084076,0.989329,0.100299,0.526129,0.182133,0.356803
6,3883,"[0.3140328672030615, 0.3953703289473061]",89,3794,1914,13616,23240,54,4.522358,10.294118,7.266260,73.350336,66.084076,0.972561,0.123245,0.630562,0.218768,0.408939
7,3882,"[0.39540905029487433, 0.5133337340191325]",208,3674,1825,9822,27034,143,10.569106,9.968526,17.835366,83.318863,66.084076,0.927337,0.156693,0.733503,0.268087,0.467493
8,3882,"[0.5133664461952903, 0.6867788670460662]",450,3432,1617,6148,30708,351,22.865854,9.311917,40.701220,92.630779,66.084076,0.821646,0.208242,0.833189,0.332272,0.517043


>> **Catboost in Validation sample**

In [181]:
df_validation_2_cat_model = transform_2_cat_model(df_validation)
df_validation_2_cat_model['probabilty'] = modelo_cat.predict_proba(df_validation_2_cat_model)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_validation_2_cat_model, probability, label, quantil)

C:\Users\caanp\AppData\Local\Temp\ipykernel_35136\2181454435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_validation_2_cat_model['probabilty'] = modelo_cat.predict_proba(df_validation_2_cat_model)[:,1]


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,2059,"[0.0011636988187688913, 0.016893957961887383]",47,2012,940,19647,0,0,5.000000,10.240749,5.000000,10.240749,34.107627,1.000000,0.045660,0.000000,0.087332,0.193042
1,2059,"[0.01690002034297615, 0.026985074588606304]",49,2010,893,17635,2012,47,5.212766,10.230570,10.212766,20.471319,34.107627,0.950000,0.048197,0.102407,0.091740,0.200332
2,2058,"[0.026985102296139936, 0.06331812571514304]",57,2001,844,15625,4022,96,6.063830,10.184761,16.276596,30.656080,34.107627,0.897872,0.051248,0.204713,0.096961,0.208611
3,2059,"[0.06337877894579586, 0.15613940834580825]",60,1999,787,13624,6023,153,6.382979,10.174581,22.659574,40.830661,34.107627,0.837234,0.054611,0.306561,0.102534,0.216554
4,2059,"[0.15615385665976464, 0.2257414166108376]",48,2011,727,11625,8022,213,5.106383,10.235659,27.765957,51.066321,34.107627,0.773404,0.058857,0.408307,0.109389,0.225608
5,2058,"[0.22580498676771635, 0.29941761192841243]",52,2006,679,9614,10033,261,5.531915,10.210210,33.297872,61.276531,34.107627,0.722340,0.065967,0.510663,0.120894,0.241585
6,2059,"[0.2994964025337689, 0.3743311068163837]",56,2003,627,7608,12039,313,5.957447,10.194941,39.255319,71.471471,34.107627,0.667021,0.076138,0.612765,0.136676,0.261359
7,2058,"[0.37451669083031364, 0.4826761822578908]",77,1981,571,5605,14042,369,8.191489,10.082964,47.446809,81.554436,34.107627,0.607447,0.092455,0.714715,0.160483,0.287339
8,2059,"[0.4826812738182562, 0.6597566128877154]",125,1934,494,3624,16023,446,13.297872,9.843742,60.744681,91.398178,34.107627,0.525532,0.119961,0.815544,0.195334,0.313531


## Using period in evaluation

> **modelo MELI**

Apresenta uma relativa estabilidade quando se olha os score para as semanas

In [198]:
df_full = pd.concat([df,df_validation])

In [214]:
## Ordenando as datas disponíveis no dataset
datas = np.sort( df_full['data'].unique() )
## total de dias para cada grupo
days = 10
## Total de grupos
total_groups = int(len(datas)/days) if int(len(datas)/days) == len(datas)/days else int(len(datas)/days+1)
## Lista de arrays contendo os dias para cada grupo
ls_group = []

i = 0
for x in range(1,total_groups+1):

    # print(x)
    group = datas[i:i+days]
    ls_group.append(group)

    i = i+days

print(f"Total de grupos:", total_groups)

Total de grupos: 5


In [229]:
probability = "score"
label="fraude"
quantil=10

@interact

def meli_model( i  = range( len(ls_group) ) ):

    print(f"Grupo {i}: ", ls_group[i], "\n\n")

    df_marco = df_full[df_full['data'].isin(ls_group[i])]

    display(utils_ml.metric_evaluation(df_marco, probability, label, quantil))

interactive(children=(Dropdown(description='i', options=(0, 1, 2, 3, 4), value=0), Output()), _dom_classes=('w…

> **Catboost**

Apresenta uma relativa estabilidade no tempo quando se olha para as semanas

In [230]:
probability = "probabilty"
label="fraude"
quantil=10

@interact

def cat_model( i  = range( len(ls_group) ) ):

    print(f"Grupo {i}: ", ls_group[i], "\n\n")

    df_marco_cat = transform_2_cat_model( df_full[df_full['data'].isin(ls_group[i])] )
    df_marco_cat['probabilty'] = modelo_cat.predict_proba(df_marco_cat)[:,1]

    display(utils_ml.metric_evaluation(df_marco_cat, probability, label, quantil))

interactive(children=(Dropdown(description='i', options=(0, 1, 2, 3, 4), value=0), Output()), _dom_classes=('w…

> **Logistic Regression**

Apresenta instabilidade quando se escore por semana e se olha para os decis

In [231]:
probability = "probabilty"
label="fraude"
quantil=10

@interact

def cat_model( i  = range( len(ls_group) ) ):

    print(f"Grupo {i}: ", ls_group[i], "\n\n")

    df_marco_model_dum = transform_2_model( df_full[df_full['data'].isin(ls_group[i])] , label ="fraude", cols_model = df_2_model_dum.columns.tolist())

    ######### Fitting ###########################
    df_marco_model_dum['probabilty'] = model.predict_proba( df_marco_model_dum.drop("fraude", axis=1) )[:,1]


    display(utils_ml.metric_evaluation(df_marco_model_dum, probability, label, quantil))


interactive(children=(Dropdown(description='i', options=(0, 1, 2, 3, 4), value=0), Output()), _dom_classes=('w…

In [38]:
!lsb_release -a

'lsb_release' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


# Requirements

In [110]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt

5697.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
